In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
from fractions import Fraction
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh = 8
refRatio = 2
refCells = 3

In [3]:
def OutputThings(waves):
    WTW = waves.transpose() @ waves
    det = LA.det(WTW)
    sym = WTW - WTW.transpose()
    WTWinv = LA.inv(WTW)
    syminv = WTWinv - WTWinv.transpose()
    ID1 = WTWinv @ WTW
    ID2 = WTW @ WTWinv
    eigs = LA.eigvals(WTW)
    print('')
    print('W^TW:')
    print(np.round(WTW, 14))
    print('')
    print('determinant =', det)
    print('')
    print('eigenvalues:')
    print(np.round(eigs, 14))
    print('')
    print('symmetry of W^TW:')
    print(np.round(sym, 14))
    print('')
    print('(W^TW)^-1:')
    print(np.round(WTWinv, 14))
    print('')
    print('symmetry of (W^TW)^-1:')
    print(np.round(syminv, 14))
    print('')
    print('identity 1:')
    print(np.round(ID1, 14))
    print('')
    print('identity 2:')
    print(np.round(ID2, 14))
    return

In [4]:
def DotEach(waves):
    iters = np.shape(waves)[1]
    print(np.round(waves, 14))
    print('')
    for k in range(iters):
        for j in range(k, iters):
            dotprod = np.round(waves[:, k].transpose() @ waves[:, j], 14)
            print(str(k) + ' dot ' + str(j) + ': ' + str(dotprod))
            print('')
    return

In [5]:
finehalf = list(np.arange(refCells))
omega = BT.Grid(nh)
omega.AddPatch(refRatio, finehalf)

waves = WT.MakeWaves(omega)
nullspace = OT.FindNullspace(omega, waves)
wavesMod = waves @ nullspace

The results in this cell make sense to me.

In [6]:
print('AMR Waves without Nullspace Treatment:')
OutputThings(waves)

AMR Waves without Nullspace Treatment:

W^TW:
[[11.          2.17355586  0.90031632  0.63661977 -0.63661977  0.12430774  0.30010544  0.63661977 -0.          0.07458465 -0.18006326  0.21220659  0.21220659  0.31050798 -0.12861662  0.        ]
 [ 2.17355586  5.6449278   0.3091803   0.33185799 -1.08159875 -0.34201114 -0.0614757   0.2374103  -0.03912151 -0.06559342 -0.3001708  -0.10435706 -0.03253574  0.48304994 -0.14389237 -0.22801689]
 [ 0.90031632  0.3091803   5.0265672   1.19775364  0.61228068  0.62254657 -0.34201114  0.09833857  0.09444769  0.43135765  0.06559342  0.05834457 -0.11504742  0.14389237 -0.1952652   0.55048148]
 [ 0.63661977  0.33185799  1.19775364  4.87534728  0.266213    0.56699787 -1.19825783 -0.40528473 -0.33574887  0.13462188 -0.1885054   0.67547456 -0.40528473 -0.30709629  0.39106089  0.33574887]
 [-0.63661977 -1.08159875  0.61228068  0.266213    4.87534728  1.24822428  0.4463682   0.40528473 -0.33574887 -0.03810141  0.22848549  0.40528473 -0.67547456  0.49367168  0.0

In [7]:
DotEach(waves)

[[ 1.          0.19383918  0.97449536  0.37292323  0.90031632  0.52399458  0.7842133   0.63661977  0.63661977  0.70419489  0.47052798  0.72451862  0.30010544  0.69987414  0.13921362  0.63661977]
 [ 1.          0.55200728  0.82613727  0.90031632  0.37292323  0.92504266 -0.18400243  0.63661977 -0.63661977  0.16522745 -0.83065451 -0.30010544 -0.72451862 -0.59332465 -0.39644686 -0.63661977]
 [ 1.          0.82613727  0.55200728  0.90031632 -0.37292323  0.18400243 -0.92504266 -0.63661977 -0.63661977 -0.83065451  0.16522745 -0.30010544  0.72451862  0.39644686  0.59332465  0.63661977]
 [ 1.          0.97449536  0.19383918  0.37292323 -0.90031632 -0.7842133  -0.52399458 -0.63661977  0.63661977  0.47052798  0.70419489  0.72451862 -0.30010544 -0.13921362 -0.69987414 -0.63661977]
 [ 1.          0.97449536 -0.19383918 -0.37292323 -0.90031632 -0.7842133   0.52399458  0.63661977  0.63661977  0.47052798 -0.70419489 -0.72451862 -0.30010544 -0.13921362  0.69987414  0.63661977]
 [ 1.          0.82613727

The results in this cell do not.

In [22]:
OutputThings(wavesMod[:, :-1])


W^TW:
[[11.          2.17355586  0.90031632  0.63661977 -0.63661977  0.12430774  0.30010544  0.63661977 -0.          0.        ]
 [ 2.17355586  5.6449278   0.3091803   0.33185799 -1.08159875 -0.34201114 -0.0614757   0.2374103  -0.03912151 -0.22801689]
 [ 0.90031632  0.3091803   5.0265672   1.19775364  0.61228068  0.62254657 -0.34201114  0.09833857  0.09444769  0.55048148]
 [ 0.63661977  0.33185799  1.19775364  4.87534728  0.266213    0.56699787 -1.19825783 -0.40528473 -0.33574887  0.33574887]
 [-0.63661977 -1.08159875  0.61228068  0.266213    4.87534728  1.24822428  0.4463682   0.40528473 -0.33574887  0.33574887]
 [ 0.12430774 -0.34201114  0.62254657  0.56699787  1.24822428  4.18287729  0.02328145  0.46124288 -1.4880857  -0.25531514]
 [ 0.30010544 -0.0614757  -0.34201114 -1.19825783  0.4463682   0.02328145  4.22944018  1.11353881  0.61638528  0.10575499]
 [ 0.63661977  0.2374103   0.09833857 -0.40528473  0.40528473  0.46124288  1.11353881  4.45813208  0.         -0.        ]
 [-0.    

In [23]:
print('AMR Waves with Nullspace Treatment:')
OutputThings(wavesMod)

AMR Waves with Nullspace Treatment:

W^TW:
[[11.          2.17355586  0.90031632  0.63661977 -0.63661977  0.12430774  0.30010544  0.63661977 -0.          0.         -0.        ]
 [ 2.17355586  5.6449278   0.3091803   0.33185799 -1.08159875 -0.34201114 -0.0614757   0.2374103  -0.03912151 -0.22801689 -0.11131875]
 [ 0.90031632  0.3091803   5.0265672   1.19775364  0.61228068  0.62254657 -0.34201114  0.09833857  0.09444769  0.55048148  0.26874724]
 [ 0.63661977  0.33185799  1.19775364  4.87534728  0.266213    0.56699787 -1.19825783 -0.40528473 -0.33574887  0.33574887  0.22160708]
 [-0.63661977 -1.08159875  0.61228068  0.266213    4.87534728  1.24822428  0.4463682   0.40528473 -0.33574887  0.33574887  0.22160708]
 [ 0.12430774 -0.34201114  0.62254657  0.56699787  1.24822428  4.18287729  0.02328145  0.46124288 -1.4880857  -0.25531514 -0.04605904]
 [ 0.30010544 -0.0614757  -0.34201114 -1.19825783  0.4463682   0.02328145  4.22944018  1.11353881  0.61638528  0.10575499  0.01907828]
 [ 0.6366197

In [9]:
DotEach(wavesMod)

[[ 1.          0.19383918  0.97449536  0.37292323  0.90031632  0.52399458  0.7842133   0.63661977  0.63661977  0.63661977  0.0374941 ]
 [ 1.          0.55200728  0.82613727  0.90031632  0.37292323  0.92504266 -0.18400243  0.63661977 -0.63661977 -0.63661977  0.60170122]
 [ 1.          0.82613727  0.55200728  0.90031632 -0.37292323  0.18400243 -0.92504266 -0.63661977 -0.63661977  0.63661977 -1.12296063]
 [ 1.          0.97449536  0.19383918  0.37292323 -0.90031632 -0.7842133  -0.52399458 -0.63661977  0.63661977 -0.63661977  1.12296063]
 [ 1.          0.97449536 -0.19383918 -0.37292323 -0.90031632 -0.7842133   0.52399458  0.63661977  0.63661977  0.63661977 -0.60170122]
 [ 1.          0.82613727 -0.55200728 -0.90031632 -0.37292323  0.18400243  0.92504266  0.63661977 -0.63661977 -0.63661977 -0.0374941 ]
 [ 1.          0.37292323 -0.90031632 -0.63661977  0.63661977  0.72451862 -0.30010544 -0.63661977 -0.          0.          0.        ]
 [ 1.         -0.37292323 -0.90031632  0.63661977  0.63

In [10]:
print('Nullspace:')
OutputThings(nullspace)

Nullspace:

W^TW:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]

determinant = 1.000000000000003

eigenvalues:
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

symmetry of W^TW:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

(W^TW)^-1:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0

In [11]:
DotEach(nullspace)

[[ 1.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          1.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          1.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          1.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          1.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.          1.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.  

In [18]:
print(nullspace[:, :-1])

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
